# Google Vision Cloud API FOR OCR

In [11]:
import os
from google.cloud import vision
import cv2

# Set up the environment variable for Google Cloud authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Point Detection\signature-429903-6c362955b89e.json"

# Initialize the Google Cloud Vision client
client = vision.ImageAnnotatorClient()

def detect_text(image_path):
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(
        image=image,
        image_context={"language_hints": ["zh", "en"]}
    )
    if response.error.message:
        raise Exception(f"OCR error: {response.error.message}")
    return response.text_annotations

if __name__ == "__main__":
    image_path = r"C:\Point Detection\preprocessing\raw\034c84874e949eb3a44a499f64396542.png"
    texts = detect_text(image_path)

    # 1. Make sure the output folder exists
    out_dir = r"C:\\Point Detection\\preprocessing\\ocr_output"
    os.makedirs(out_dir, exist_ok=True)

    # 2. Write OCR output to a text file in that folder
    output_file = os.path.join(out_dir, "ocr_output.txt")
    with open(output_file, 'w', encoding='utf-8') as file:
        for text in texts:
            print(text.description)
            file.write(text.description + "\n")

    print(f"Text has been written to {output_file}")


上海浦东陆家嘴西路正大广场却
ADSG822506300017
单号: A080827506300017
交易日: 2025-06-30
折扣
金额
产品
J64952
999.00
100.00% 1
999.00
00-5
2020231
0.00
0.00%
1
0.00
79-MIC
2
999.00
999.00
查收
999.00
0.00
故事
(扫码):999.00
打印:2025-06-30 21:08:32
編集:2025-08-30 21:08:11
地址:上海市演系新法家彍出道家嘴西路168号正大广场20网
迪达斯
TEL: 80411920
特别提示
招弹,创造运动好生活
1.收银小票是您获取电子发票的唯一凭证,请乏善保管
2.您可使用支付宝或者扫描下方二维码取电子发票
3.此二维码有效期为交易日起30天,服照开票,需到纳售门店家訂
申请,谢谢
4.开票申请或后电子发票将于三个工作日内发送至您填寫的电子
邮箱
5.使用卡券面額內的金額不开具发票
6.如有问题,请及时联系门店
电子发票
上海
浦东
陆家嘴
西路
正大
广场
却
ADSG822506300017
单
号
:
A080827506300017
交易
日
:
2025-06-30
折扣
金额
产品
J64952
999.00
100.00
%
1
999.00
00-5
2020231
0.00
0.00
%
1
0.00
79
-
MIC
2
999.00
999.00
查收
999.00
0.00
故事
(
扫
码
)
:
999.00
打印
:
2025-06-30
21:08:32
編集
:
2025-08-30
21:08:11
地址
:
上海市
演
系
新法
家
彍
出道
家嘴
西路
168
号
正大
广场
20
网
迪达斯
TEL
:
80411920
特别
提示
招
弹
,
创造
运动
好
生活
1.
收银
小
票
是
您
获取
电子
发票
的
唯一
凭证
,
请
乏
善
保管
2.
您
可
使用
支付
宝
或者
扫描
下
方二维
码
取
电子
发票
3.
此
二维
码
有效期
为
交易
日
起
30
天
,
服
照
开票
,
需
到
纳
售
门店
家
訂
申请
,
谢谢
4.
开票
申请
或
后
电子
发票
将于
三个
工作
日内
发送
至
您
填寫
的
电子
邮

# Use AI to find time and date and recognizes brand
Setup Model

In [12]:
from dotenv import dotenv_values
import os

# 1. Read only valid KEY=VALUE lines (ignores the rest)
config = dotenv_values(r"C:\Point Detection\.env")

# 2. Pull out your key
api_key = config.get("OPENAI_API_KEY")
if not api_key:
    raise EnvironmentError("OPENAI_API_KEY not found in .env")

# 3. Stick it into os.environ for the rest of your code
os.environ["OPENAI_API_KEY"] = api_key

# 4. Initialize your model
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [13]:
from dotenv import load_dotenv
import os, json
import openai

# 1) Load env & set API key
load_dotenv(r"C:\Point Detection\.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

# 2) Read your OCR output
ocr_path = r"C:\Point Detection\preprocessing\ocr_output\ocr_output.txt"
with open(ocr_path, encoding="utf-8") as f:
    ocr_text = f.read()

# 3) Define the function schema
functions = [
    {
        "name": "parse_summary",
        "description": "Extract store_name, receipt_number, date_time, total_price",
        "parameters": {
            "type": "object",
            "properties": {
                "store_name":     {"type": "string"},
                "receipt_number": {"type": "string"},
                "date_time":      {"type": "string", "format": "date-time"},
                "total_price":    {"type": "number"}
            },
            "required": ["store_name", "receipt_number", "date_time", "total_price"]
        }
    }
]

# 4) Call the new Chat Completions API
resp = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":"Extract key fields from a Chinese/English receipt."},
        {"role":"user",   "content":ocr_text}
    ],
    functions=functions,
    function_call={"name":"parse_summary"}
)

# 5) Pull out the JSON arguments
call = resp.choices[0].message.function_call
data = json.loads(call.arguments)

# 6) (Optional) save to file
out_path = r"C:\\Point Detection\\preprocessing\\processed\\parsed_summary.json"
with open(out_path, "w", encoding="utf-8") as out:
    json.dump(data, out, ensure_ascii=False, indent=2)

# 7) Print the result
print(json.dumps(data, ensure_ascii=False, indent=2))


{
  "store_name": "迪达斯",
  "receipt_number": "A080827506300017",
  "date_time": "2025-06-30T21:08:32",
  "total_price": 999.0
}
